# Statistics

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

pd.options.mode.chained_assignment = None

## Search strategy comparison

In [2]:
df = pd.read_json('experiments/results/nasbench201/cifar10/tier1absent/accuracies.json')
print(df.mean())
print(ttest_ind(df.mtnas, df.re, alternative='greater'))
print(ttest_ind(df.mtnas, df.rs, alternative='greater'))
print()

df = pd.read_json('experiments/results/nasbench201/cifar10/tier2absent/accuracies.json')
print(df.mean())
print(ttest_ind(df.mtnas, df.re, alternative='greater'))
print(ttest_ind(df.mtnas, df.rs, alternative='greater'))

mtnas    91.426250
re       91.241500
rs       90.842125
dtype: float64
Ttest_indResult(statistic=6.042331524711918, pvalue=5.2559089977953775e-09)
Ttest_indResult(statistic=17.877307026359638, pvalue=4.312771850302285e-40)

mtnas    91.280625
re       91.241500
rs       90.842125
dtype: float64
Ttest_indResult(statistic=1.1070503891769659, pvalue=0.13497736015456394)
Ttest_indResult(statistic=11.796581338527181, pvalue=9.780309661115345e-24)


## Averages

In [3]:
def per_epoch_averages(paths):
    dfs = list()
    for path in paths:
        df = pd.read_json(path).dropna(axis=1, how='all')
        df.columns = pd.MultiIndex.from_tuples(df.columns.map(eval)).rename(['algorithm', 'seed'], level=[0, 1])
        df.index.rename('epoch', inplace=True)
        df = df.stack([0, 1]).groupby(['epoch', 'algorithm']).mean().unstack(1).drop(('estm', 're'), axis=1)
        df.true.re[50:50 + df.true.re.dropna().shape[0]] = df.true.re.dropna()
        df.true.re[:50] = np.nan
        dfs.append(df)
    return dfs

In [4]:
t2h, dflt = per_epoch_averages(('experiments/results/nasbench201/cifar10/tier2high/averages.json',
                                'experiments/results/nasbench201/cifar10/default/averages.json'))

In [5]:
ttest_ind(t2h.true.mtnas, dflt.true.mtnas)

Ttest_indResult(statistic=0.25740103551671756, pvalue=0.7969574542314518)